In [7]:
import numpy as np
from numba import jit
import time 

In [8]:
@jit(nopython=True)
def legendre(n,x):
    leg = 1.
    if n == 0:
        leg = 1
    elif n==1:
        leg = x
    else:
        leg_down1 = x; leg_down2 = 1.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            leg_down2 = leg_down1
            leg_down1 = leg

    return leg

@jit(nopython=True)
def dlegendre(n,x):
    dleg = 0.
    if n == 0:
        dleg = 0.
    elif n == 1:
        dleg = 1.
    else:
        leg_down1 = x; leg_down2 = 1.
        dleg_down1 = 1.; dleg_down2 = 0.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            dleg = dleg_down2 + (2*i-1)*leg_down1
            leg_down2 = leg_down1
            leg_down1 = leg
            dleg_down2 = dleg_down1
            dleg_down1 = dleg

    return dleg

@jit(nopython=True)
def gauss_legendre_lobatto(length:int):
    #// define some constants
    tolerance = 4.0 * 1.0e-6
    nnewton_iter = 100
    pi = np.pi
    
    # allocate space
    n = length - 1; #// order of polynomial
    x = np.array([0.0 for i in range(length)])
    w = x * 1.
    if n == 1:
        x[0] = -1.; x[1] = 1.
        w[0] = 1.; w[1] = 1.
    else:
        leg = 0.; dleg = 0.; delta = 0.
        # set end points
        x[0]   = -1.0; x[n] = 1.
        w[0]   =  2./(n*(n+1.)); w[n] =  2./(n*(n+1.))

        for i in range(1,(n+1)//2):
            #// initial guess from an approximate form given by SV Parter (1999)
            x[i] = -np.cos( (i+0.25)*pi/n  - 3/(8*n*pi*(i+0.25)))

            #// newton iteration
            for j in range(nnewton_iter):
                leg = legendre(n+1,x[i]) - legendre(n-1,x[i])
                dleg = dlegendre(n+1,x[i]) - dlegendre(n-1,x[i])
                delta = -leg/dleg
                x[i] += delta
                if (np.abs(delta) <= tolerance * np.abs(x[i]) ): break
            
            x[n-i] = - x[i]
            leg = legendre(n, x[i])
            w[i] = 2./(n*(n+1.)*leg*leg)
            w[n-i] = w[i]

        if n %2 == 0 :
            x[n//2] = 0.
            leg = legendre(n, 0.0)
            w[n//2]  = 2./(n*(n+1.)*leg*leg)
    

    return x,w

@jit(nopython=True)
def lagrange_poly(xi,xctrl):
    nctrl = len(xctrl)
    hprime = np.array([0.0 for i in range(nctrl)])
    h = hprime * 1.0

    #! note: this routine is hit pretty hard by the mesher, optimizing the loops here will be beneficial
    for dgr in range(nctrl):
        prod1 = 1.; prod2 = 1.

        #// lagrangian interpolants
        x0 = xctrl[dgr]
        for i in range(nctrl):
            if i != dgr:
                x = xctrl[i]
                prod1 = prod1*(xi-x)
                prod2 = prod2*(x0-x)

        #//! takes inverse to avoid additional divisions
        #//! (multiplications are cheaper than divisions)
        prod2_inv = 1. / prod2
        h[dgr] = prod1 * prod2_inv

        #// first derivatives
        s = 0.0
        for i in range(nctrl):
            if i != dgr :
                prod3 = 1.0
                for j in range(nctrl):
                    if j != dgr and j != i:
                        prod3 = prod3*(xi-xctrl[j])
                s = s + prod3
        hprime[dgr] = s * prod2_inv
    

    return h,hprime


def init_gll_arrays(NGLLX):

    # gll points and weights
    xgll,wxgll = gauss_legendre_lobatto(NGLLX)

    #// derivative, hprimex[i][j] = l_j'(xi_i)
    hprimex = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        _,hprimex[i,:] = lagrange_poly(xgll[i],xgll)

    #// hprimex_wgllx[i,j] = hprime_x[i,j] * wx[i] 
    hprimex_wgllx = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        for j in range(NGLLX):
            hprimex_wgllx[i,j] = hprimex[i,j] * wxgll[i]


    return xgll,wxgll,hprimex,hprimex_wgllx



In [9]:
def create_sem_mesh(zmax,nel,NGLL):
    # connectivity matrix
    ibool = np.zeros((nel,NGLL),dtype=int)
    idx = 0
    for i in range(nel):
        for j in range(NGLL):
            ibool[i,j] = idx 
            idx += 1
        idx -= 1
    
    # jacobians
    jaco = np.zeros((nel))
    jaco[:] = zmax / nel * 0.5 

    return ibool,jaco 
    

In [10]:
@jit(nopython=True)
def assemble_matrix(P,ibool,wgll,hprime,jaco,):
    nspec,NGLL = P.shape
    nglob = np.max(ibool) + 1
    E = np.zeros((nglob,nglob),dtype=float)
    hpT = hprime.transpose()

    for ispec in range(nspec):
        sum_terms=  P[ispec,:] * wgll[:] / jaco[ispec]
        
        for i in range(NGLL):
            ig1 = ibool[ispec,i]
            for j in range(NGLL):
                ig2 = ibool[ispec,j]
                s = np.sum(sum_terms * hpT[i,:] * hpT[j,:])
                E[ig1,ig2] += s 
    
    return E 

@jit(nopython=True)
def assemble_matrix_fd(ibool,wgll,hprime,jaco,x,y,P):
    nspec,NGLL = ibool.shape 
    dm = np.zeros((nspec,NGLL),dtype=float)
    for ispec in range(nspec):
        for i in range(NGLL):
            p = P[ispec,i] * 1. 
            dx = 0.01
            P[ispec,i] = p * (1 + dx)
            E = assemble_matrix(P,ibool,wgll,hprime,jaco)
            s1 = y.dot(E.dot(x))

            P[ispec,i] = p * (1 - dx)
            E = assemble_matrix(P,ibool,wgll,hprime,jaco)
            s2 = y.dot(E.dot(x))

            dm[ispec,i] = (s1 - s2) / (p * dx * 2)

            # recover
            P[ispec,i] = p * 1. 
    
    return dm 
@jit(nopython=True)
def assemble_matrix_deriv(ibool,wgll,hprime,jaco,x,y):
    nspec,NGLL = ibool.shape
    dE = np.zeros((nspec,NGLL))

    # element
    xe = np.zeros((NGLL),dtype=float)
    ye = np.zeros((NGLL),dtype=float)

    for ispec in range(nspec):
        # cache element
        for i in range(NGLL):
            ig = ibool[ispec,i]
            xe[i] = x[ig]
            ye[i] = y[ig]

        # compute 
        sum_terms = wgll[:] / jaco[ispec]
        for k in range(NGLL):
            s1 = np.sum(hprime[k,:] * xe[:])
            s2 = np.sum(hprime[k,:] * ye[:])
            dE[ispec,k] = sum_terms[k] * s1 * s2 
    
    return dE

In [17]:
@jit(nopython=True)
def assemble_matrix_diag(P,ibool,wgll,jaco):
    nspec,NGLL = P.shape
    nglob = np.max(ibool) + 1
    M = np.zeros((nglob),dtype=float)

    for ispec in range(nspec):
        for i in range(NGLL):
            ig = ibool[ispec,i]
            M[ig] += jaco[ispec] * wgll[i] * P[ispec,i] 
    
    return M 

@jit(nopython=True)
def assemble_matrix_diag_fd(ibool,wgll,jaco,x,y,P):
    nspec,NGLL = ibool.shape 
    dm = np.zeros((nspec,NGLL),dtype=float)
    for ispec in range(nspec):
        for i in range(NGLL):
            p = P[ispec,i] * 1. 
            dx = 0.01
            P[ispec,i] = p * (1 + dx)
            M = assemble_matrix_diag(P,ibool,wgll,jaco)
            s1 = np.sum(y * M * x)

            P[ispec,i] = p * (1 - dx)
            M = assemble_matrix_diag(P,ibool,wgll,jaco)
            s2 = np.sum(y * M * x)

            dm[ispec,i] = (s1 - s2) / (p * dx * 2)

            # recover
            P[ispec,i] = p * 1. 
    
    return dm 

@jit(nopython=True)
def assemble_matrix_diag_deriv(ibool,wgll,jaco,x,y):
    nspec,NGLL = ibool.shape
    dM = np.zeros((nspec,NGLL),dtype=float)

    # element
    xe = np.zeros((NGLL),dtype=float)
    ye = np.zeros((NGLL),dtype=float)

    for ispec in range(nspec):
        # cache element
        for i in range(NGLL):
            ig = ibool[ispec,i]
            xe[i] = x[ig]
            ye[i] = y[ig]

        # compute 
        for k in range(NGLL):
            dM[ispec,k] =  wgll[k] * jaco[ispec] * xe[k] * ye[k]
    
    return dM

In [21]:
nspec = 400
NGLL = 5
xgll,wgll,hprime,_ = init_gll_arrays(NGLL)
ibool,jaco = create_sem_mesh(11.5,nspec,NGLL)
nglob = np.max(ibool) + 1

# random seed
np.random.seed(15)
P = np.random.rand(nspec,NGLL)
x = np.random.rand(nglob)
y = np.random.rand(nglob)

tic = time.time()
dm = assemble_matrix_diag_fd(ibool,wgll,jaco,x,y,P)
toc = time.time()
print('time = ',toc-tic)

tic = time.time()
dE = assemble_matrix_diag_deriv(ibool,wgll,jaco,x,y)
toc = time.time()
print('time = ',toc-tic)

print(np.allclose(dm,dE))

time =  0.020659208297729492
time =  5.316734313964844e-05
True


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

plt.figure(1,figsize=(14,6))
Q = np.logspace(np.log10(3),np.log10(500),200)
y = np.sqrt(1 + 1j/ Q)
y1 = 1 + 1j / (2 * Q) 

plt.subplot(121)
plt.semilogx(Q,y.real)
plt.semilogx(Q,y1.real)
plt.subplot(122)
plt.semilogx(Q,y.imag)
plt.semilogx(Q,y1.imag)

In [ ]:
import numpy as np 
def get_Q_sls_model(Q):

    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref * 0
    NSLS = len(y)

    for i in range(NSLS):
        y[i] = y_sls_ref[i] / Q 
    dy[0] = 1. + 0.5 * y[0]

    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]

    #// copy to y_sls/w_sls
    w_sls = w_sls_ref * 1. 
    y_sls = dy * y 

    return w_sls,y_sls 


def compute_q_sls_model(y_sls,w_sls,om,exact=False):
    Q_ls = 1. 
    nsls = len(y_sls)
    if exact:
        for p in range(nsls):
            Q_ls += y_sls[p] * om**2 / (om**2 + w_sls[p]**2)

    # denom
    Q_demon = 0.
    for p in range(nsls):
        Q_demon += y_sls[p] * om * w_sls[p] / (om**2 + w_sls[p]**2)
    
    return Q_ls / Q_demon

def get_sls_modulus_factor(freq,Q):
    om = 2 * np.pi * freq

    w_sls,y_sls = get_Q_sls_model(Q)
    s = np.sum(1j * om * y_sls / (w_sls + 1j * om))

    return s + 1.

def get_sls_Q_deriv(freq,Q):
    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref  / Q 

    # corrector
    NSLS = len(y_sls_ref)
    dy[0] = 1. + 0.5 * y[0]
    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]
    dd_dqi = dy * 0
    dd_dqi[0] = 0.5 * y_sls_ref[0]
    for i in range(1,NSLS):
        dd_dqi[i] = dd_dqi[i-1] + (dy[i-1] - 0.5) * y_sls_ref[i-1] + dd_dqi[i-1] * y[i-1] +  0.5 * y_sls_ref[i]
        
    dd_dqi = dd_dqi * y + dy * y_sls_ref
    om = 2 * np.pi * freq
    dsdqi = np.sum(1j * om * dd_dqi /(w_sls_ref + 1j * om))
    s = np.sum(1j * om * y * dy / (w_sls_ref + 1j * om))

    return s + 1., dsdqi

In [ ]:
Q = 50.
freq=  1.
s,dsdqi = get_sls_Q_deriv(freq,Q)

dQ = 0.001
s1 = get_sls_modulus_factor(freq,Q * (1 + dQ))
s2 = get_sls_modulus_factor(freq,Q * (1-dQ))
print(s,s1,s2)
dsdq = (s1 - s2) / (Q * dQ * 2)
dsdqi_fdf = -Q**2 * dsdq

print(dsdqi,dsdqi_fdf)

In [1]:
import sympy as sp 

In [7]:
L2,L1,N2,N1,rho2,rho1,om,H,c = sp.symbols("L2,L1,N2,N1,r2,r1,om,H,c")
sn1,sn2,sl1,sl2,dsdqn1,dsdqn2,dsdql1,dsdql2 = sp.symbols("sn1,sn2,sl1,sl2,dsdqn1,dsdqn2,dsdql1,dsdql2")
f = L2/L1 * sp.sqrt((N2/L2) - (rho2 * c**2)/L2) - sp.sqrt((c**2 *rho1)/L1 - (N1/L1)) * sp.tan((om * H)/c *sp.sqrt((c**2 * rho1)/L1 - (N1/L1)))

gamma2 = sp.sqrt((1 - rho2 * c**2/N2) * N2/L2)
Omega = om/c * H * gamma2 * (
            ((rho1 * c**2 - N1) / (N2 - rho2 / rho1 * N1)) +
            (L2 / L1) * ((N2 - rho2*c**2) / (N2 - rho2/rho1 * N1))
        )
u = N1 / (c * rho1) * ( c**2 * rho1 / N1 + Omega) / ( 1 + Omega)

In [8]:
all_params = (L2,L1,N2,N1,rho2,rho1)
for param in all_params:
    outstr = sp.pycode(-sp.diff(f,param) / sp.diff(f,c)).replace("math.","np.")
    print(f"dcc_dc{str(param)}={outstr}")

dcc_dcL2=(-L2*(-1/2*N2/L2**2 + (1/2)*c**2*r2/L2**2)/(L1*np.sqrt(N2/L2 - c**2*r2/L2)) - np.sqrt(N2/L2 - c**2*r2/L2)/L1)/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
dcc_dcL1=(H*om*((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/c + ((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/np.sqrt(-N1/L1 + c**2*r1/L1) + L2*np.sqrt(N2/L2 - c**2*r2/L2)/L1**2)/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
dcc_dcN2=-(1/2)/(L1*np.sqrt(N2

In [9]:
all_params = (L2,L1,N2,N1,rho2,rho1)
for param in all_params:
    expr = sp.diff(u,param)  - sp.diff(u,c) * sp.diff(f,param) / sp.diff(f,c)
    outstr = sp.pycode(expr).replace("math.","np.")
    print(f"dcu_dc{str(param)}={outstr}")

dcu_dcL2=N1*(-1/2*H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(L2*c) + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)/(L1*c*(-N1*r2/r1 + N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*((1/2)*H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(L2*c) - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)/(L1*c*(-N1*r2/r1 + N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - (L2*(-1/2*N2/L2**2 + (1/2)*c**2*r2/L2**2)/(L1*np.sqrt(N2/L2 - c**2*r2/L2)) + np.sqrt(N2/L2 - c**2*r2/L2)/L1)*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) 

In [ ]:
sp.diff(f,c)